In [50]:
QRY_PATH="/home/global/anaconda2/tract_querier/queries/freesurfer_queries.qry"
#SUBJ_ID="T02S01"
SUBJ_ID="p_AR28"
NAS="tupac"
#protocole="healthy_volunteers"
protocole="alexcis"
DIR_PATH="/NAS/"+NAS+"/protocoles/"+protocole
FS_PATH=DIR_PATH+"/FS53/"
DTI_PATH=FS_PATH+SUBJ_ID+"/dti/"
TRACTO="whole_brain_6_1500000"
#SEG = DTI_PATH+"raparc+aseg.nii"
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"
MESH = "lh.white"

In [47]:
from nibabel import trackvis as tv
from dipy.viz import fvtk
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import AveragePointwiseEuclideanMetric
from dipy.segment.metric import CenterOfMassFeature
from dipy.segment.metric import EuclideanMetric
from dipy.segment.metric import IdentityFeature
from dipy.segment.clustering import QuickBundles
from dipy.io.image import load_nifti, save_nifti
from dipy.tracking.streamline import set_number_of_points
from nibabel import freesurfer as fs
from dipy.utils.optpkg import optional_package
import dipy.io.vtk as io_vtk
import dipy.viz.utils as ut_vtk
from dipy.viz import window, actor, ui
from dipy.tracking import utils
import nipype.interfaces.mrtrix as mrt
from dipy.tracking.streamline import select_random_set_of_streamlines
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

In [217]:
!tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.trk" -q $QRY_PATH -o $DTI_PATH"result"

Loading files
Calculating labels and crossings
Traceback (most recent call last):
  File "/home/global/anaconda2/bin/tract_querier", line 286, in <module>
    main()
  File "/home/global/anaconda2/bin/tract_querier", line 185, in main
    tracts, img, affine_ijk_2_ras, options.length_threshold, options.threshold
  File "/home/global/anaconda2/lib/python2.7/site-packages/tract_querier/tract_label_indices.py", line 76, in __init__
    self.tractography, self.length_threshold, self.crossing_threshold
  File "/home/global/anaconda2/lib/python2.7/site-packages/tract_querier/tract_label_indices.py", line 245, in compute_tract_label_indices
    all_points_ijk = (np.dot(affine_ras_2_ijk[:-1, :-1], all_points.T).T +
MemoryError


In [40]:
!tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.trk" -q $QRY_PATH"wmql_1_cst.qry" -o $DTI_PATH"result"
!tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.trk" -q $QRY_PATH"wmql_2_uf.qry" -o $DTI_PATH"result"
!tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.trk" -q $QRY_PATH"wmql_3_commissural.qry" -o $DTI_PATH"result"

Loading files
Calculating labels and crossings
Computing queries
	Query cortico_spinal.left: 003694
	Query cortico_spinal.right: 003112
Loading files
Calculating labels and crossings
Computing queries
	Query uncinate.left: 000797
	Query uncinate.right: 001261
Loading files
Calculating labels and crossings
Computing queries
	Query all_commissural: 167194


In [6]:
TRACTO="result_cortico_spinal."
streams, hdr = tv.read(DTI_PATH+TRACTO+"left.trk")
cst_left = [i[0] for i in streams]
streams, hdr = tv.read(DTI_PATH+TRACTO+"right.trk")
cst_right = [i[0] for i in streams]

TRACTO="result_uncinate."
streams, hdr = tv.read(DTI_PATH+TRACTO+"left.trk")
uf_left = [i[0] for i in streams]
streams, hdr = tv.read(DTI_PATH+TRACTO+"right.trk")
uf_right = [i[0] for i in streams]
    
TRACTO="result_all_commissural"
streams, hdr = tv.read(DTI_PATH+TRACTO+".trk")
commissural = [i[0] for i in streams]

TRACTO="result_cc1_rostrum"
streams, hdr = tv.read(DTI_PATH+TRACTO+".trk")
cc1_rostrum = [i[0] for i in streams]

#TRACTO="whole_brain_6_1500000"
#streams, hdr = tv.read(DTI_PATH+TRACTO+".trk")
#whole_brain = [i[0] for i in streams]

In [7]:
feature = ResampleFeature(nb_points=240)
metric = AveragePointwiseEuclideanMetric(feature=feature)  # a.k.a. MDF
qb = QuickBundles(threshold=10, metric=metric)
qb.max_nb_clusters = 1
clusters_cc1_rostrum = qb.cluster(cc1_rostrum)
print clusters_cc1_rostrum.centroids[0][1]

[ 112.05557251  186.32392883   45.33963013]


In [8]:
r = fvtk.ren()
fvtk.add(r, fvtk.streamtube(cc1_rostrum,fvtk.colors.blue,linewidth=0.2,opacity=0.1));
fvtk.add(r, fvtk.streamtube(clusters_cc1_rostrum.centroids,linewidth=1));
#fvtk.add(r, fvtk.streamtube(cst_right,fvtk.colors.blue,linewidth=0.2,opacity=0.1));
#fvtk.add(r, fvtk.streamtube(uf_left,fvtk.colors.green,linewidth=0.2,opacity=0.1));
#fvtk.add(r, fvtk.streamtube(uf_right,fvtk.colors.green,linewidth=0.2,opacity=0.1));
#fvtk.add(r, fvtk.streamtube(commissural,fvtk.colors.red,linewidth=0.2,opacity=0.1));
#fvtk.add(r, fvtk.streamtube(whole_brain,fvtk.colors.white,linewidth=0.2,opacity=0.1));
fvtk.show(r)

In [10]:
!ls $QRY_PATH/results

fs_af.left.trk			   fs_slf_ii.left.trk
fs_af.right.trk			   fs_slf_ii.right.trk
fs_cb.left.trk			   fs_slf_i.left.trk
fs_cb.right.trk			   fs_slf_i.right.trk
fs_cc_1.trk			   fs_soff.left.trk
fs_cc_2.trk			   fs_soff.right.trk
fs_cc_3.trk			   fs_striato_frontal.left.trk
fs_cc_4.trk			   fs_striato_frontal.right.trk
fs_cc_7.trk			   fs_striato_occipital.left.trk
fs_cst.left.trk			   fs_striato_occipital.right.trk
fs_cst.right.trk		   fs_striato_parietal.left.trk
fs_ec.left.trk			   fs_striato_parietal.right.trk
fs_ec.right.trk			   fs_thalamo_frontal.left.trk
fs_extreme_capsule.left.trk	   fs_thalamo_frontal.right.trk
fs_extreme_capsule.right.trk	   fs_thalamo_occipital.left.trk
fs_ilf.left.trk			   fs_thalamo_occipital.right.trk
fs_ilf.right.trk		   fs_thalamo_parietal.left.trk
fs_internal_capsule_new.left.trk   fs_thalamo_parietal.right.trk
fs_internal_capsule_new.right.trk  fs_uf.left.trk
fs_ioff.left.trk		   fs_uf.right.trk
fs_ioff.right.trk		   JHU_cg.left.trk
fs_mdlf.left.trk		  

In [51]:
vtk, have_vtk, setup_module = optional_package('vtk')
white_mesh=fs.read_geometry(FS_PATH+SUBJ_ID+"/surf/"+MESH)
my_polydata = vtk.vtkPolyData()
ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])

#file_name = DTI_PATH+"white_matter.vtk"
#io_vtk.save_polydata(my_polydata, file_name)
#print("Surface saved in " + file_name)

#streams_whole_brain, hdr_whole_brain = tv.read(DTI_PATH+"whole_brain_6_1500000.trk")
#whole_brain = [i[0] for i in streams_whole_brain]

(vtkCommonDataModelPython.vtkPolyData)0x7fedeeb94c18

In [41]:
print(hdr_whole_brain)
plot_whole_brain = select_random_set_of_streamlines(whole_brain, 1000)

streams_LongueFibre, hdr_LongueFibre = tv.read('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk')
LongueFibre = [i[0] for i in streams_LongueFibre]
plot_LongueFibre = select_random_set_of_streamlines(LongueFibre, 100)
print(hdr_LongueFibre)

streams_fs_cst, hdr_fs_cst = tv.read("/home/romain/Téléchargements/wmql_example_dataset/results/fs_cst.left.trk")
fs_cst = [i[0] for i in streams_fs_cst]
plot_fs_cst = select_random_set_of_streamlines(fs_cst, 100)
print(hdr_fs_cst)


#print(tv.aff_to_hdr(hdr_LongueFibre))
input_spacetmp=np.array(tv.aff_from_hdr(hdr_LongueFibre));
input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])
test = utils.move_streamlines(plot_whole_brain,input_space)
test=list(test)

r = fvtk.ren()
fvtk.add(r, fvtk.streamtube(plot_whole_brain,fvtk.colors.white,linewidth=0.1,opacity=0.1));
fvtk.add(r, fvtk.streamtube(LongueFibre,fvtk.colors.red,linewidth=0.5,opacity=0.5));
fvtk.add(r, fvtk.streamtube(fs_cst,fvtk.colors.blue,linewidth=0.1,opacity=0.1));
fvtk.add(r, fvtk.streamtube(test,fvtk.colors.green,linewidth=0.2,opacity=0.2));
r.add(stream_actor)
fvtk.show(r)

('TRACK', [128, 128,  66], [ 2.,  2.,  2.], [ 0.,  0.,  0.], 0, ['', '', '', '', '', '', '', '', '', ''], 0, ['', '', '', '', '', '', '', '', '', ''], [[  -2.        ,    0.        ,    0.        ,  128.34214783], [   0.        ,    2.        ,    0.        , -129.92842102], [   0.        ,    0.        ,    2.        ,  -58.10770035], [   0.        ,    0.        ,    0.        ,    1.        ]], '', 'LAS', '', [ 0.,  0.,  0.,  0.,  0.,  0.], '', '', '', '', '', '', '', 0, 2, 1000)
('TRACK', [128, 128,  66], [ 2.,  2.,  2.], [ 0.,  0.,  0.], 0, ['', '', '', '', '', '', '', '', '', ''], 0, ['', '', '', '', '', '', '', '', '', ''], [[  -2.        ,    0.        ,    0.        ,  128.34214783], [   0.        ,    2.        ,    0.        , -129.92842102], [   0.        ,    0.        ,    2.        ,  -58.10770035], [   0.        ,    0.        ,    0.        ,    1.        ]], '', 'LAS', '', [ 0.,  0.,  0.,  0.,  0.,  0.], '', '', '', '', '', '', '', 0, 2, 1000)
('TRACK', [128, 128,  66

/mnt/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Defaulting to `atleast_v2` of False.  Future versions will default to True
  app.launch_new_instance()


# Association tract name
cb=Cingular/cingulum Bundle 
ec=extreme capsule
extreme_capsule=extreme capsule
internal_capsule_new=internal capsule
af=Arcuate fascicle
slf=superior longitudinal fasiculus 1,2 et 3
ilf=inferior longitudinal fasiculus
mdlf=middle longitudinal fasiculus
ioff=inferior occipito-frontal fasiculus
soff=superior occipito-frontal fasiculus
uf=uncinate fasiculus

# Commisural tract name
cc_1=Rostrum
cc_2=Genu
cc_3=Rostral body
cc_4=Anterior mid-body
cc_5=Posterior mid-body
cc_6=Isthmus
cc_7=Splenium

# Projection tract name
cst=cortico spinal tract


In [52]:
r = fvtk.ren()
li = [ 'fs_af.left','fs_slf_ii.left','fs_af.right','fs_slf_ii.right','fs_cb.left','fs_slf_i.left','fs_cb.right','fs_slf_i.right','fs_cc_1','fs_soff.left','fs_cc_2','fs_soff.right','fs_cc_3','fs_striato_frontal.left','fs_cc_4','fs_striato_frontal.right','fs_cc_7','fs_striato_occipital.left','fs_cst.left','fs_striato_occipital.right','fs_cst.right','fs_striato_parietal.left','fs_ec.left','fs_striato_parietal.right','fs_ec.right','fs_thalamo_frontal.left','fs_extreme_capsule.left','fs_thalamo_frontal.right','fs_extreme_capsule.right','fs_thalamo_occipital.left','fs_ilf.left','fs_thalamo_occipital.right','fs_ilf.right','fs_thalamo_parietal.left','fs_internal_capsule_new.left','fs_thalamo_parietal.right','fs_internal_capsule_new.right','fs_uf.left','fs_ioff.left','fs_uf.right','fs_ioff.right','fs_mdlf.left','fs_mdlf.right','fs_slf_iii.left','fs_slf_iii.right' ]
li = [ 'fs_af.left','fs_slf_ii.left','fs_af.right','fs_slf_ii.right','fs_cb.left','fs_slf_i.left','fs_cb.right','fs_slf_i.right','fs_cc_1','fs_soff.left','fs_cc_2','fs_soff.right','fs_cc_3']

#li = ['fs_uf.right','fs_uf.left'] 

data_seg, affine = load_nifti(SEG)
vol_actor = fvtk.slicer(data_seg)
vol_actor.display(50, None, None)
vol_actorBis = fvtk.slicer(data_seg)
vol_actorBis.display(40, None, None)

print(affine)

data_seg2, affine2 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/aparc_aseg_native_ras.nii.gz")
vol_actor2 = fvtk.slicer(data_seg2)
vol_actor2.display(150, None, None)
print(affine2)

data_seg3, affine3 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/mri/aparc+aseg.nii")
vol_actor3 = fvtk.slicer(data_seg3)
vol_actor3.display(120, None, None)
print(affine3)

#TRACTO="whole_brain_6_1500000"
#streams, hdr = tv.read(DTI_PATH+TRACTO+".trk")
#whole_brain = [i[0] for i in streams]
#fvtk.add(r, fvtk.streamtube(whole_brain,fvtk.colors.white,linewidth=0.1,opacity=0.1));
input_spacetmp=np.array(tv.aff_from_hdr(hdr_LongueFibre));
input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])
print(affine)
#print(input_spacetmp)

for TRACTO in li:
    streams, hdr = tv.read("/home/romain/Téléchargements/wmql_example_dataset/results/"+TRACTO+".trk")
    #input_spacetmp=np.array(tv.aff_from_hdr(hdr));
    #input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])

    tracto = [i[0] for i in streams]
    feature = ResampleFeature(nb_points=240)
    metric = AveragePointwiseEuclideanMetric(feature=feature)  # a.k.a. MDF
    qb = QuickBundles(threshold=10, metric=metric)
    qb.max_nb_clusters = 1
    cluster = qb.cluster(tracto)
    #print cluster[0][1]
    
    plot_tracto = select_random_set_of_streamlines(tracto, 1000)
    plot_tracto=list(utils.move_streamlines(plot_tracto,input_space))
    fvtk.add(r, fvtk.streamtube(plot_tracto,fvtk.colors.blue,linewidth=0.1,opacity=1));
    test=list(utils.move_streamlines(cluster.centroids,input_space))
    fvtk.add(r, fvtk.streamtube(test,linewidth=1));


#test = utils.move_streamlines(plot_whole_brain,input_space);
#fvtk.add(r, fvtk.streamtube(test,fvtk.colors.green,linewidth=0.2,opacity=0.2));

fvtk.add(r, vol_actor)
fvtk.add(r, vol_actorBis)
fvtk.add(r, vol_actor2)
fvtk.add(r, vol_actor3)
    
#stream_vtk = io_vtk.load_polydata("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/white_matter.vtk")
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.5) 
r.add(stream_actor)

a=fvtk.axes()
fvtk.add(r,a)
fvtk.show(r)

[[  -2.            0.            0.          128.34214783]
 [   0.            2.            0.         -129.92842102]
 [   0.            0.            2.          -58.10770035]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.84214783]
 [   0.            1.            0.         -118.78250122]
 [   0.            0.            1.         -142.00848389]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.84214783]
 [   0.            0.            1.         -118.78250122]
 [   0.           -1.            0.          113.99151611]
 [   0.            0.            0.            1.        ]]
[[  -2.            0.            0.          128.34214783]
 [   0.            2.            0.         -129.92842102]
 [   0.            0.            2.          -58.10770035]
 [   0.            0.            0.            1.        ]]


/mnt/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Defaulting to `atleast_v2` of False.  Future versions will default to True


In [155]:
data_seg2, affine = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/aparc_aseg_native_ras.nii.gz")
input_space1=np.array([[1.,0.,0.,-affine[0,3]],[0.,1.,0.,affine[1,3]],[0.,0.,1.,affine[2,3]],[0.,0.,0.,1.]])
input_space2=np.array([[1.,0.,0.,0],[0.,1.,0.,0],[0.,0.,1.,0],[0.,0.,0.,1.]])
streams, hdr = tv.read("/home/romain/Téléchargements/wmql_example_dataset/results/fs_uf.left.trk")
tracto = [i[0] for i in streams]
#plot_tracto = select_random_set_of_streamlines(tracto, 1000)
plot_tracto=list(utils.move_streamlines(tracto,input_space1))
M, grouping = utils.connectivity_matrix(tracto,data_seg2,affine=input_space2,return_mapping=True,mapping_as_streamlines=True)
#M[:3, :] = 0
#M[:, :3] = 0
print(len(tracto))
print(np.sum(M))
print(M)
plt.imshow(data_seg2[62,:,:], interpolation='nearest')
plt.savefig("connectivity.png")

474
481
[[467   0   1 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  1   0   0 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


MemoryError: 

In [19]:
data_seg, affine = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/raparc.a2009s+aseg.nii")
data_seg1, affine1 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/raparc.a2009s+aseg4mrtrix.nii")
data_seg2, affine2 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/aparc_aseg_native_ras.nii.gz")
data_seg3, affine3 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/mri/aparc+aseg.nii")
data_seg4, affine4 = load_nifti("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/mri/aparc.a2009s+aseg.nii")

print(data_seg2.shape)
print(affine)
print(affine1)
print(affine2)
print(affine3)
print(affine4)

/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/raparc.a2009s+aseg.nii
(256, 256, 256)
[[  -2.            0.            0.          128.34214783]
 [   0.            2.            0.         -129.92842102]
 [   0.            0.            2.          -58.10770035]
 [   0.            0.            0.            1.        ]]
[[  -2.            0.            0.          128.34214783]
 [   0.            2.            0.         -129.92842102]
 [   0.            0.            2.          -58.10770035]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.84214783]
 [   0.            1.            0.         -118.78250122]
 [   0.            0.            1.         -142.00848389]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.84214783]
 [   0.            0.            1.         -118.78250122]
 [   0.           -1.            0.          113.99151611]
 [   0.            0.      

In [38]:
#tck2trk = mrt.MRTrix2TrackVis()
#tck2trk.inputs.in_file = "/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.tck"
#tck2trk.inputs.image_file ="/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/b0.nii"
#tck2trk.inputs.out_filename = "/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk"
#tck2trk.run()

r = fvtk.ren()
vtk, have_vtk, setup_module = optional_package('vtk')

#my_polydata = io_vtk.load_polydata("/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/white_matter.vtk")
white_mesh=fs.read_geometry('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/surf/lh.white.ras')
my_polydata = vtk.vtkPolyData()
ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.5) 
r.add(stream_actor)
print(my_polydata.GetBounds())

streams, hdr = tv.read('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk')
tracto = [i[0] for i in streams]
print(tracto[0][0])
print(tracto[0][152])
#fvtk.add(r, fvtk.streamtube(tracto,fvtk.colors.red,linewidth=1,opacity=1));

input_space1=np.array([[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]);
input_space2=np.array([[0.4605,-0.8867,0.0415,102.8482],[0.0706,-0.0100,-0.9975,-37.4021],[0.8848,0.4623,0.0580,62.2839],[0.,0.,0.,1.]]);

test4 = utils.move_streamlines(tracto,input_space1)
test5 = utils.move_streamlines(test4,input_space2)
test5=list(test5)
fvtk.add(r, fvtk.streamtube(test5,fvtk.colors.red,linewidth=1,opacity=1));
print(test5[0][0])
print(test5[0][152])

#fvtk.add(r, fvtk.streamtube(whole_brain,fvtk.colors.white,linewidth=0.1,opacity=0.1));
plot_whole_brain = select_random_set_of_streamlines(whole_brain, 1000)
fvtk.add(r, fvtk.streamtube(plot_whole_brain,fvtk.colors.blue,linewidth=0.4,opacity=0.4));

tmp_plot_whole_brain = utils.move_streamlines(plot_whole_brain,input_space1)
test_plot_whole_brain = utils.move_streamlines(tmp_plot_whole_brain,input_space2)
test_plot_whole_brain=list(test_plot_whole_brain)
fvtk.add(r, fvtk.streamtube(test_plot_whole_brain,fvtk.colors.white,linewidth=0.4,opacity=0.4));

stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.5) 
r.add(stream_actor)

a=fvtk.axes()
fvtk.add(r,a)

fvtk.show(r, size=(600, 600), reset_camera=False)

(-62.662742614746094, 3.7138798236846924, -87.10131072998047, 73.38294982910156, -40.64234924316406, 63.65271759033203)
[ 124.92929077   48.86561584   34.44001389]
[ 147.1055603    61.67776489   19.89728165]
[  3.41838061 -81.06467794 -23.66544146]
[-18.75784746 -68.25206865 -38.20742671]


In [221]:
print(hdr)

output_filename = '/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres2.tck'
trk = nib.streamlines.load('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti//uneLongueFibres.trk')
print(tracto[0][1])
print(nib.streamlines.detect_format('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk'))
print(nib.streamlines.detect_format('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/whole_brain_6_1500000.tck'))
print(nib.streamlines.detect_format('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.tck'))
print(nib.streamlines.detect_format('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres2.tck'))

nib.streamlines.save(trk.tractogram,output_filename)

#input_space=np.array([[-0.3417,0.9103,-0.2335,190.3189],[-0.9128,-0.2623,0.3131,36.1256],[0.2237,0.3201,0.9206,82.8371],[0.,0.,0.,1.]]);
input_space=np.array([[-1.,0.,-0.,10],[-0.,1.,0.,50.],[0.,0.,1.,100.],[0.,0.,0.,1.]]);
#test = utils.move_streamlines(tracto,[[1.,0.,0.,2.],[0.,2.,0.,1.],[0.,0.,2.,1.],[0.,0.,0.,1.]], input_space=[[-0.3417,0.9103,-0.2335,190.3189], [-0.9128,-0.2623,0.3131,36.1256], [0.2237,0.3201,0.9206,82.8371], [0.,0.,0.,1.]])
test = utils.move_streamlines(tracto,input_space)
test=list(test)
fvtk.add(r, fvtk.streamtube(test,fvtk.colors.green,linewidth=0.2,opacity=1));

input_space=np.array([[0.958757698535919,-0.197806343436241,-0.204100653529167,-118.0309829711914],[-0.250771343708038,-0.926710844039917,-0.279859095811844,15.9356689453125],[-0.133785068988800,0.319499671459198,-0.938095867633820,-3.00267076492310],[0.,0.,0.,1.]])
test2 = utils.move_streamlines(tracto,input_space)
test2=list(test2)
print(test2[0][1])
print(test2[0][152])
fvtk.add(r, fvtk.streamtube(test2,fvtk.colors.red,linewidth=0.2,opacity=1));


trackvis_point_space = utils.affine_for_trackvis([-0.5,-0.5,-0.5])
print(tracto)
test = utils.move_streamlines(tracto,trackvis_point_space,input_space=np.eye(4))
#input_space=[[-2.,0.,0.,128.34214783], [0.,2.,0.,-129.92842102], [0.,0.,2.,-58.10770035], [0.,0.,0.,1.]])
test=list(test)
fvtk.add(r, fvtk.streamtube(test,fvtk.colors.white,linewidth=0.2,opacity=1));


input_space=np.array([[1.,0.,0,-121.6318],[0.,1.,0.,-130.0751],[0.,0.,1.,-58.0358],[0.,0.,0.,1.]]);
test3 = utils.move_streamlines(tracto,input_space)
test3=list(test3)
fvtk.add(r, fvtk.streamtube(test3,fvtk.colors.pink,linewidth=0.2,opacity=1));


#MRview = mrt.MRTrixViewer()
#MRview.inputs.in_files='/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/b0.nii'
#MRview.run()

('TRACK', [128, 128,  66], [ 2.,  2.,  2.], [ 0.,  0.,  0.], 0, ['', '', '', '', '', '', '', '', '', ''], 0, ['', '', '', '', '', '', '', '', '', ''], [[  -2.        ,    0.        ,    0.        ,  128.34214783], [   0.        ,    2.        ,    0.        , -129.92842102], [   0.        ,    0.        ,    2.        ,  -58.10770035], [   0.        ,    0.        ,    0.        ,    1.        ]], '', 'LAS', '', [ 0.,  0.,  0.,  0.,  0.,  0.], '', '', '', '', '', '', '', 0, 2, 1000)
[ 125.04470825   49.01226807   34.36809158]
<class 'nibabel.streamlines.trk.TrkFile'>
None
None
None


ValueError: Unknown tractogram file format: '/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres2.tck'

In [261]:
white_mesh=fs.read_geometry('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/surf/lh.white.ras')
my_polydata = vtk.vtkPolyData()
ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.5) 
r.add(stream_actor)
print(my_polydata.GetBounds())

white_mesh=fs.read_geometry('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/surf/lh.white')
my_polydata = vtk.vtkPolyData()
ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.5) 
r.add(stream_actor)
print(my_polydata.GetBounds())

(-62.662742614746094, 3.7138798236846924, -87.10131072998047, 73.38294982910156, -40.64234924316406, 63.65271759033203)
(-64.50489044189453, 1.8717319965362549, -96.31880950927734, 64.16545104980469, -26.63386344909668, 77.66120147705078)


In [14]:
tractoNibabel = nib.streamlines.load('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti//uneLongueFibres.trk')
print(tractoNibabel)

streams, hdr = tv.read('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk')
print hdr

tracto = [i[0] for i in streams]
print(tracto[0][0])
print(tracto[0][152])

trackvis_point_space = utils.affine_for_trackvis([2,2,2])
test = utils.move_streamlines(tracto,trackvis_point_space, input_space=[[-2.,0.,0.,128.34214783], [0.,2.,0.,-129.92842102], [0.,0.,2.,-58.10770035], [0.,0.,0.,1.]])
test=list(test)
#print(test[0][0])

input_space=np.array([[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]);
testtmp=utils.move_streamlines(tracto,input_space)
testtmp=list(testtmp)
print(testtmp[0][0])
print(testtmp[0][152])

input_space=np.array([[0.4605,-0.8867,0.0415,102.8482],[0.0706,-0.0100,-0.9975,-37.4021],[0.8848,0.4623,0.0580,62.2839],[0.,0.,0.,1.]]);
#test = utils.move_streamlines(tracto,[[1.,0.,0.,2.],[0.,2.,0.,1.],[0.,0.,2.,1.],[0.,0.,0.,1.]], input_space=[[-0.3417,0.9103,-0.2335,190.3189], [-0.9128,-0.2623,0.3131,36.1256], [0.2237,0.3201,0.9206,82.8371], [0.,0.,0.,1.]])
test4=utils.move_streamlines(testtmp,input_space)
test4=list(test4)
print(test4[0][0])
print(test4[0][152])


NameError: name 'nib' is not defined

In [63]:
trackvis_point_space = utils.affine_for_trackvis([1,1,1])
test = utils.move_streamlines(tracto,trackvis_point_space, input_space=[[-0.3417,0.9103,-0.2335,190.3189], [-0.9128,-0.2623,0.3131,36.1256], [0.2237,0.3201,0.9206,82.8371], [0.,0.,0.,1.]])
liste=list(test)
print(liste)

[array([[  3.85050868e-01,  -7.78646475e+01,  -2.47952947e+01],
       [  1.95657642e-01,  -7.78210712e+01,  -2.48425507e+01],
       [  1.72838417e-02,  -7.77623332e+01,  -2.49113557e+01],
       [ -1.65311682e-01,  -7.77056817e+01,  -2.49700931e+01],
       [ -3.31485348e-01,  -7.76144644e+01,  -2.50338712e+01],
       [ -5.03196288e-01,  -7.75179166e+01,  -2.50684386e+01],
       [ -6.60219929e-01,  -7.74045597e+01,  -2.51184031e+01],
       [ -8.07931724e-01,  -7.72875720e+01,  -2.51854509e+01],
       [ -9.70738980e-01,  -7.71974106e+01,  -2.52586966e+01],
       [ -1.12683501e+00,  -7.70863809e+01,  -2.53162156e+01],
       [ -1.29118071e+00,  -7.69978468e+01,  -2.53879995e+01],
       [ -1.47098226e+00,  -7.69410699e+01,  -2.54547150e+01],
       [ -1.63894251e+00,  -7.68708248e+01,  -2.55375186e+01],
       [ -1.80448008e+00,  -7.67872387e+01,  -2.56124389e+01],
       [ -1.95135696e+00,  -7.66958888e+01,  -2.57128587e+01],
       [ -2.10755342e+00,  -7.66155593e+01,  -2.580852

In [ ]:
##MATLAB
tck=f_readFiber_tck('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.tck')
tck_pt=[tck.fiber.xyzFiberCoord(1,:) 1; tck.fiber.xyzFiberCoord(153,:) 1]'

trk=f_readFiber_trk('/NAS/tupac/protocoles/alexcis/FS53/p_AR28/dti/uneLongueFibres.trk')
surf_lh='/NAS/tupac/protocoles/alexcis/FS53/p_AR28/surf/lh.white.ras';
trk_pt=[ 124.92929077   48.86561584   34.44001389 1 ; 147.1055603    61.67776489   19.89728165 1]'

dist(tck_pt)
dist(trk_pt)

%vox_to_world=[[  -2.            0.            0.          128.34214783]
% [   0.            2.            0.         -129.92842102]
% [   0.            0.            2.          -58.10770035]
% [   0.            0.            0.            1.        ]]

%test=vox_to_world * trk_pt;
%dist(test)

%[TR1, TT1, ER1] = icp(trk_pt(1:3,:),tck_pt(1:3,:))
%TR1 * tck_pt(1:3,1) + TT1 

vox_to_world=[[  -1.            0.            0.          0]
 [   0.            1.            0.         0]
 [   0.            0.            1.          0]
 [   0.            0.            0.            1.        ]]

trk_pt2=vox_to_world * trk_pt;

[TR2, TT2, ER2] = icp(tck_pt(1:3,:),trk_pt2(1:3,:))
[TR2 , TT2] * trk_pt2(:,1) 

surf = SurfStatReadSurf({[surf_lh]});
minmax(surf.coord)
trk_pt
[TR2 , TT2] * trk_pt2
tck_pt

figure;
plot3(tck.fiber.xyzFiberCoord(:,1),tck.fiber.xyzFiberCoord(:,2),tck.fiber.xyzFiberCoord(:,3),'r')
hold on;
plot3(surf.coord(1,1:10:138869)',surf.coord(2,1:10:138869)',surf.coord(3,1:10:138869)','.')

trk_cst=f_readFiber_trk('/home/romain/Téléchargements/wmql_example_dataset/results/fs_cst.left.trk')